In [2]:
library(devtools)
library("fgsea")
library(devtools)
library(tidyverse)
library(Matrix)

── Attaching packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.5 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
Warning message:
“package ‘ggplot2’ was built under R version 4.2.1”
Warning message:
“package ‘tibble’ was built under R version 4.2.1”
Warning message:
“package ‘stringr’ was built under R version 4.2.1”
── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack




In [25]:
convertTerms <- function(x){
    out <- lapply(seq(1:nrow(x)), function(z){
        gids <- strsplit(as.character(x$V2[z]), "\\,")
        gids <- do.call(c, gids)
        return(gids)
    })
    names(out) <- as.character(x$V1)
    return(out)
}

In [35]:
read_table_run_FGSEA <- function(de_seq2_input, location, functional_annotations){
    
    input_file_name <- here(location, de_seq2_input)
    sb_denovo <- read.table(input_file_name,
                          header = TRUE)
    
    sb_denovo.selected <- sb_denovo  %>% 
        dplyr::select(gene_name, stat)

    ranks <- deframe(sb_denovo.selected)
    
    fgseaRes <- fgsea(pathways=functional_annotations, stats=ranks, 
                  nperm=10000,
                 minSize = 10, maxSize = 600)
    
    fgseaResTidy <- fgseaRes %>%
        as_tibble()
        
    
    fgseaResTidy$celltype <- de_seq2_input

    return(fgseaResTidy)
}

In [18]:
pathways <- read_delim("/home/jpm73279/genome_downloads/Sbicolor.v5.1.prelim.annot/Sbicolorv5.1.GO_terms_from_ZM.txt", delim = "\t" ,
                      col_names = c("geneID", "term", "GOID"))
pathway_format <- pathways  %>% 
    dplyr::select(geneID, term)  %>% 
    distinct()  %>% 
    group_by(term)  %>% 
    mutate(COUNTER = 1:n()) %>%
    ungroup() %>% 
    group_by(term)  %>% 
    summarise(funcion = paste(geneID, collapse=","))  %>% 
    rename(V2 = funcion)  %>% 
    rename(V1 = term) 
    

Rows: 74877 Columns: 3
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): geneID, term, GOID

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [50]:
x <- as.data.frame(pathway_format)
head(x)

,V1,V2
,<chr>,<chr>
1,'de novo' AMP biosynthetic process,"Sobic.001G486700,Sobic.001G126300"
2,'de novo' cotranslational protein folding,Sobic.006G049900
3,'de novo' CTP biosynthetic process,Sorbiv5.1_pg8597.valid
4,'de novo' IMP biosynthetic process,"Sobic.006G247900,Sobic.001G114400,Sorbiv5.1_pg6149,Sobic.006G083100"
5,'de novo' pyrimidine nucleobase biosynthetic process,Sobic.004G318100
6,(-)-E-beta-caryophyllene synthase activity,Sobic.001G369200


In [51]:
sb_functional_annotation <- convertTerms(as.data.frame(pathway_format))


In [29]:
library("here")

here() starts at /home/jpm73279/Mendieta_et_al_comparative_single_cell



In [54]:
str(sb_functional_annotation)

List of 3418
 $ 'de novo' AMP biosynthetic process                                                                                                                                   : chr [1:2] "Sobic.001G486700" "Sobic.001G126300"
 $ 'de novo' cotranslational protein folding                                                                                                                            : chr "Sobic.006G049900"
 $ 'de novo' CTP biosynthetic process                                                                                                                                   : chr "Sorbiv5.1_pg8597.valid"
 $ 'de novo' IMP biosynthetic process                                                                                                                                   : chr [1:4] "Sobic.006G247900" "Sobic.001G114400" "Sorbiv5.1_pg6149" "Sobic.006G083100"
 $ 'de novo' pyrimidine nucleobase biosynthetic process                                                                  

In [55]:

sb_de_seq2 <- c("Sb.V4.final_annotation.bundle_sheath_ncell_1919.upregulated_genes.deseq2_output.tsv",
"Sb.V4.final_annotation.companion_cells_sieve_elements_ncell_591.upregulated_genes.deseq2_output.tsv",
"Sb.V4.final_annotation.epidermis_ncell_3830.upregulated_genes.deseq2_output.tsv",
"Sb.V4.final_annotation.ground_meristem_ncell_880.upregulated_genes.deseq2_output.tsv",
"Sb.V4.final_annotation.mesophyll_ncell_3997.upregulated_genes.deseq2_output.tsv",
"Sb.V4.final_annotation.procambium_ncell_591.upregulated_genes.deseq2_output.tsv",
"Sb.V4.final_annotation.protoderm_ncell_2290.upregulated_genes.deseq2_output.tsv",
"Sb.V4.final_annotation.proto_xylem_ncell_494.upregulated_genes.deseq2_output.tsv",
"Sb.V4.final_annotation.xylem_ncell_800.upregulated_genes.deseq2_output.tsv")

In [56]:
working_location <- "/scratch/jpm73279/comparative_single_cell/06.de_novo_id_markers"
all_gathered_cell_type_FGSEA <- lapply(sb_de_seq2, read_table_run_FGSEA, location = working_location, functional_annotations = sb_functional_annotation)

Warning message in fgsea(pathways = functional_annotations, stats = ranks, nperm = 10000, :
“You are trying to run fgseaSimple. It is recommended to use fgseaMultilevel. To run fgseaMultilevel, you need to remove the nperm argument in the fgsea function call.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.07% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“All values in the stats vector are greater than zero and scoreType is "std", maybe you should switch to scoreType = "pos".”
Warning message in fgsea(pathways = functional_annotations, stats = ranks, nperm = 10000, :
“You are trying to run fgseaSimple. It is recommended to use fgseaMultilevel. To run fgseaMultilevel, you need to remove the nperm argument in the fgsea function call.”
Warning message in prep

In [57]:
sb_cell_type_FGSEA <- bind_rows(all_gathered_cell_type_FGSEA)

In [58]:
fitlered_save <- sb_cell_type_FGSEA  %>% 
    group_by(celltype)  %>% 
    mutate_at(c('pval'), as.numeric)  %>% 
    dplyr::filter(NES > 0 & pval < .05) %>% 
    dplyr::arrange(pval, descending = TRUE)  %>% 
    dplyr::arrange(celltype)  %>% 
    mutate(celltype = str_replace(celltype, "Sb.V4.final_annotation.", ""))  %>% 
    mutate(celltype = str_replace(celltype, ".upregulated_genes.deseq2_output.tsv", ""))  %>% 
    dplyr::select(-leadingEdge)

write_delim(fitlered_save, "Sb.FGSEA_results.tsv", delim = "\t")